In [112]:
import pickle
import MeCab

In [113]:
def load_file(name):
    with open("wordtable/"+name+".pickle", "rb") as f:
        return pickle.load(f)

In [114]:
word_table = load_file("word_table")
page_title_id_table = load_file("page_title_id_table")
page_heading_id_table = load_file("page_heading_id_table")
page_desc_id_table = load_file("page_desc_id_table")
zenbun_table = load_file("zenbun_table")

In [115]:
def get_word_id(word):
    global word_table
    if word not in word_table["data"]:
        return None
    return word_table["data"][word]

In [116]:
wakati = MeCab.Tagger("-Owakati")
remove_words = {"(", ")", "（", "）", "[", "]",
                    "「", "」", "+", "-", "*", "$",
                    "'", '"', "、", ".", "”", "’",
                    ":", ";", "_", "/", "?", "!",
                    "。", ",", "=", "＝"}
def split_word(keyword):
    return [get_word_id(r) for r in wakati.parse(keyword).split() if r not in remove_words]

In [117]:
def set_score(result, word_id, table, score):
    if word_id not in table: 
        return 
    pageset = table[word_id]
    for page in pageset:
        if page not in result:
            result[page] = 0
        result[page] += score
def zenbun_search(result,keyword, score):
    global zenbun_table
    for page_id, zenbun in zenbun_table.items():
        if keyword in zenbun:
            if page_id not in result:
                result[page_id] = 0
            result[page_id] += score

In [118]:
def scored_search(keyword):
    result = {}
    for word_id in split_word(keyword):
        if word_id is not None:
            set_score(result,word_id, page_title_id_table, 30)
            set_score(result,word_id, page_heading_id_table, 10)
            set_score(result,word_id, page_desc_id_table, 1)
        zenbun_search(result, keyword, 1)
    return result

In [119]:
page_dict = {}
with open("pages.pickle", "rb") as f:
    for page in pickle.load(f):
        page_dict[page["id"]] = page

In [120]:
def sort_score(scores):
    score_array = []
    for page_id, score in scores.items():
        score_array.append({
            "page_id": page_id,
            "score": score
        })
    score_array.sort(key=lambda x: x["score"])
    return score_array

In [121]:
def search(keyword):
    if len(keyword) == 0:
        return
    scores = scored_search(keyword)
    scores = [page_dict[s["page_id"]] for s in sort_score(scores)]
    return scores
    

In [123]:
search("テスト")

[{'id': 477,
  'date': '2021-04-02T00:39:34',
  'date_gmt': '2021-04-01T15:39:34',
  'guid': {'rendered': 'https://www.stb.tsukuba.ac.jp/~shinkan-web/orgadmin/?page_id=477'},
  'modified': '2021-04-02T00:39:34',
  'modified_gmt': '2021-04-01T15:39:34',
  'slug': '',
  'status': 'draft',
  'type': 'page',
  'link': 'https://www.stb.tsukuba.ac.jp/~shinkan-web/orgadmin/?page_id=477',
  'title': {'rendered': 'The Empire of TSUKUBA'},
  'content': {'rendered': '<p><strong>ヲタ芸・サイリウムダンスをするサークル。</strong></p>\n<p>2018年に春日エリアの学生たちによって設立されました。<br />\n初代メンバーが数人脱退し、現在４年生４人、３年生２人、２年生２人で活動しています。</p>\n<p>ヲタ芸自体に興味を持って大学からヲタ芸を始めた人やヲタ芸経験者まで幅広く所属しています。</p>\n<p>練習日は週１～２回を予定しています。主に日曜日は固定。（テスト一週間前とテスト期間は練習なし）</p>\n<p><a href="https://www.youtube.com/channel/UCHxYTEgJwElet2l5bY0MH5A/featured">雙峰祭や地域のイベントなどでパフォーマンスをしたり、YouTubeに動画を投稿したりします。</a></p>\n<p>いろんなジャンルのオタクが打ってます！ヲタ芸でプロオタクになろう！（？）</p>\n<p>P.S. Kpopオタクからアニオタからボカロオタクなど幅広いジャンルのオタクがいます。</p>\n',
   'protected': False},
  'excerpt': {'rendered': '<p>ヲタ芸・サイリウ